<center><b>Funciones para Endspoints (API)<b><center>

In [1]:
# Importacion de librerias
import pandas as pd
import ast
import pdb

In [2]:
# Carga de datasets
df_items = pd.read_parquet('..\\datasets\\New_datasets\\user_items.parquet')
df_reviews = pd.read_parquet('..\\datasets\\New_datasets\\user_reviews_sentiments.parquet')
df_steams = pd.read_csv('..\\datasets\\New_datasets\\user_steams.csv')

C:\Users\DELL\AppData\Local\Temp\ipykernel_10508\1214550425.py:4: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_steams = pd.read_csv('..\\datasets\\New_datasets\\user_steams.csv')


In [3]:
df_items['steam_id'] = df_items['steam_id'].apply(float)
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          float64
 3   item_id           object 
 4   item_name         object 
 5   playtime_forever  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 235.9+ MB


<b>PlayTimeGenre:</b>       

Debe devolver año con mas horas jugadas para dicho género.

In [4]:
# Se eliminan los datos nulos de las columnas a consultar
df_steams_genre = df_steams.dropna(subset='genres')
df_steams_genre.reset_index(inplace=True,drop=True)
df_steams_genre = df_steams_genre.dropna(subset='release_year')
df_steams_genre.reset_index(inplace=True,drop=True)
df_steams_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28660 entries, 0 to 28659
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     23751 non-null  object 
 1   genres        28660 non-null  object 
 2   app_name      28659 non-null  object 
 3   title         28659 non-null  object 
 4   url           28660 non-null  object 
 5   release_date  28660 non-null  object 
 6   tags          28636 non-null  object 
 7   reviews_url   28659 non-null  object 
 8   specs         28374 non-null  object 
 9   price         27615 non-null  object 
 10  early_access  28660 non-null  object 
 11  id            28659 non-null  float64
 12  developer     28490 non-null  object 
 13  release_year  28660 non-null  float64
dtypes: float64(2), object(12)
memory usage: 3.1+ MB


In [5]:
# Desanidado de columna genres de dataset df_steams_genre
datos = []
def genre_reader(df_genre):
    try:
        for index,line in enumerate(df_genre['genres']):
            dato_genres = ast.literal_eval(line) 
            #pdb.set_trace()
            for genre in dato_genres:
                #pdb.set_trace()
                datos.append([genre,df_genre['release_year'][index],df_genre['app_name'][index],df_genre['title'][index]])
    except:
        pdb.set_trace()

    return pd.DataFrame(datos,columns=['genres','release_year','app_name','title'])

In [6]:
# Se observa que se presentan columna duplicadas
df_PlayTimeGenre = genre_reader(df_steams_genre)
# Se trabaja con una de las columnas nombre y se rescata solo las columnas de interes
df_PlayTimeGenre = df_PlayTimeGenre.merge(df_items,left_on='title', right_on='item_name')
df_PlayTimeGenre = df_PlayTimeGenre[['genres','release_year','playtime_forever']]

# Se agrupa 'genres','release_year' y se suma la columna 'playtime_forever' segun lo solicitado. 
# Se utiliza la funcion transform para especificar el contenido de la nueva columna
df_PlayTimeGenre['Sum_playtime_forever'] = df_PlayTimeGenre.groupby(['genres','release_year'])['playtime_forever'].transform('sum')

# Se elimina la columna 'playtime_forever' que ya se utilizo para la suma y se quita duplicados generados en group by
df_PlayTimeGenre.drop(columns='playtime_forever',inplace=True)
df_PlayTimeGenre.drop_duplicates(subset=['genres','release_year'],inplace=True)
df_PlayTimeGenre.reset_index(drop=True,inplace=True)
df_PlayTimeGenre.info()
df_PlayTimeGenre.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                341 non-null    object 
 1   release_year          341 non-null    float64
 2   Sum_playtime_forever  341 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 8.1+ KB


,genres,release_year,Sum_playtime_forever
0,Action,1997.0,271013
1,Indie,1997.0,64814
2,Racing,1997.0,9319


In [7]:
# Funcion de consulta por genero el año de mayor cantidad de horas acumuladas
def PlayTimeGenre(genre):
    anio = df_PlayTimeGenre['release_year'][df_PlayTimeGenre['Sum_playtime_forever'][df_PlayTimeGenre['genres'] == genre].idxmax()]
    return int(anio)

In [8]:
df_PlayTimeGenre[df_PlayTimeGenre['genres'] == 'Action'][df_PlayTimeGenre['genres'] == 'Action']

C:\Users\DELL\AppData\Local\Temp\ipykernel_10508\2605478342.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_PlayTimeGenre[df_PlayTimeGenre['genres'] == 'Action'][df_PlayTimeGenre['genres'] == 'Action']


,genres,release_year,Sum_playtime_forever
0,Action,1997.0,271013
3,Action,1998.0,2809535
5,Action,2005.0,6015794
13,Action,2000.0,18553775
17,Action,2007.0,11309542
19,Action,1996.0,51726
20,Action,1994.0,40603
21,Action,2001.0,1255120
22,Action,2003.0,9906989
24,Action,1999.0,2721215


In [9]:
# Prueba
PlayTimeGenre('Action')

2012

<b>UserForGenre:</b>       

Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año. Es similar la funcion PlayTimeGenre, pero se agrega usuario y lista de juegos con horas jugadas

In [10]:
# Consulta de generos desanidados y merge con dataframe items
df_UserForGenre = genre_reader(df_steams_genre)
df_UserForGenre = df_UserForGenre.merge(df_items,left_on='title', right_on='item_name')
df_UserForGenre = df_UserForGenre[['genres','user_id','release_year','playtime_forever']]


In [11]:
# Agrupado segun lo requerido
df_UserForGenre_group = df_UserForGenre.copy()
df_UserForGenre_group['Sum_playtime_year'] = df_UserForGenre_group.groupby(['genres','user_id','release_year'])['playtime_forever'].transform('sum')
df_UserForGenre_group.drop(columns='playtime_forever',inplace=True)
df_UserForGenre_group.drop_duplicates(subset=['genres','user_id','release_year'],inplace=True)
df_UserForGenre_group.reset_index(drop=True,inplace=True)

In [12]:
df_UserForGenre_group.head(3)

,genres,user_id,release_year,Sum_playtime_year
0,Action,UTNerd24,1997.0,2386
1,Action,I_DID_911_JUST_SAYING,1997.0,0
2,Action,76561197962104795,1997.0,0


In [13]:
df_genres = df_UserForGenre_group['genres']
df_genres.drop_duplicates(inplace=True)
df_genres.reset_index(drop=True,inplace=True)
df_genres

0                        Action
1                         Indie
2                        Racing
3                      Strategy
4                    Simulation
5                        Casual
6                     Adventure
7                           RPG
8                        Sports
9                  Free to Play
10        Massively Multiplayer
11                 Early Access
12                    Education
13            Software Training
14                    Utilities
15               Web Publishing
16    Design &amp; Illustration
17     Animation &amp; Modeling
18                Photo Editing
19             Audio Production
20             Video Production
Name: genres, dtype: object

In [14]:
# Funcion de agrupamiento y consulta
def UserForGenre(genre):
    # Captando el maximo de tiempo
    user = df_UserForGenre_group['user_id'][df_UserForGenre_group['Sum_playtime_year'][df_UserForGenre_group['genres'] == genre].idxmax()]
    lista = df_UserForGenre_group[df_UserForGenre_group['genres'] == genre][df_UserForGenre_group['user_id'] == user]
    lista = lista[['release_year','Sum_playtime_year']]
    lista.sort_values(by = ['release_year'], inplace = True)
    lista.reset_index(drop = True,inplace = True)
    return {'user' : user, 'lista' : lista}

In [15]:
UserForGenre('Indie')

C:\Users\DELL\AppData\Local\Temp\ipykernel_10508\2284509799.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lista = df_UserForGenre_group[df_UserForGenre_group['genres'] == genre][df_UserForGenre_group['user_id'] == user]


{'user': 'REBAS_AS_F-T',
 'lista':     release_year  Sum_playtime_year
 0         1999.0                  0
 1         2001.0                 22
 2         2003.0               3726
 3         2005.0                  0
 4         2006.0               3346
 5         2007.0                  0
 6         2008.0               2732
 7         2009.0              54060
 8         2010.0              42972
 9         2011.0             169974
 10        2012.0             241880
 11        2013.0             326132
 12        2014.0             559860
 13        2015.0            1439870
 14        2016.0            1550174
 15        2017.0              64322}

In [43]:

def UserForGenre_total():
    return {genre: UserForGenre(genre) for genre in df_genres}
df_UserForGenre_Total = UserForGenre_total()

C:\Users\DELL\AppData\Local\Temp\ipykernel_10508\2284509799.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lista = df_UserForGenre_group[df_UserForGenre_group['genres'] == genre][df_UserForGenre_group['user_id'] == user]
C:\Users\DELL\AppData\Local\Temp\ipykernel_10508\2284509799.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lista = df_UserForGenre_group[df_UserForGenre_group['genres'] == genre][df_UserForGenre_group['user_id'] == user]
C:\Users\DELL\AppData\Local\Temp\ipykernel_10508\2284509799.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lista = df_UserForGenre_group[df_UserForGenre_group['genres'] == genre][df_UserForGenre_group['user_id'] == user]
C:\Users\DELL\AppData\Local\Temp\ipykernel_10508\2284509799.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lista = df_UserForGenre_group[df_UserForGenre_group['genres'] == genre][df_UserFor

In [45]:
df_UserForGenre_Total.get('Indie')

{'user': 'REBAS_AS_F-T',
 'lista':     release_year  Sum_playtime_year
 0         1999.0                  0
 1         2001.0                 22
 2         2003.0               3726
 3         2005.0                  0
 4         2006.0               3346
 5         2007.0                  0
 6         2008.0               2732
 7         2009.0              54060
 8         2010.0              42972
 9         2011.0             169974
 10        2012.0             241880
 11        2013.0             326132
 12        2014.0             559860
 13        2015.0            1439870
 14        2016.0            1550174
 15        2017.0              64322}

In [55]:
df_UserForGenre = pd.DataFrame(df_UserForGenre_Total)
df_UserForGenre

,Action,Indie,Racing,Strategy,Simulation,Casual,Adventure,RPG,Sports,Free to Play,...,Early Access,Education,Software Training,Utilities,Web Publishing,Design &amp; Illustration,Animation &amp; Modeling,Photo Editing,Audio Production,Video Production
user,Evilutional,REBAS_AS_F-T,DownSyndromeKid,shinomegami,clawbot44,idonothack,REBAS_AS_F-T,Evilutional,1silentsniper1,idonothack,...,76561197978756659,76561198059330972,Lickidactyl,76561198073642113,76561198035718256,76561198035718256,76561198059330972,Colidos,Lickidactyl,Lickidactyl
lista,release_year Sum_playtime_year 0 20...,release_year Sum_playtime_year 0 ...,release_year Sum_playtime_year 0 ...,release_year Sum_playtime_year 0 ...,release_year Sum_playtime_year 0 20...,release_year Sum_playtime_year 0 ...,release_year Sum_playtime_year 0 ...,release_year Sum_playtime_year 0 20...,release_year Sum_playtime_year 0 20...,release_year Sum_playtime_year 0 20...,...,release_year Sum_playtime_year 0 20...,release_year Sum_playtime_year 0 20...,release_year Sum_playtime_year 0 20...,release_year Sum_playtime_year 0 20...,release_year Sum_playtime_year 0 20...,release_year Sum_playtime_year 0 20...,release_year Sum_playtime_year 0 20...,release_year Sum_playtime_year 0 20...,release_year Sum_playtime_year 0 20...,release_year Sum_playtime_year 0 20...


<b>UsersRecommend:</b>       

Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

In [19]:
df_reviews_recomend = df_reviews.dropna(subset='recommend')
df_reviews_recomend.reset_index(drop=True,inplace=True)
df_reviews_recomend.info()
df_reviews_recomend.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49186 entries, 0 to 49185
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             49186 non-null  object
 1   item_id             49186 non-null  object
 2   recommend           49186 non-null  bool  
 3   review              49186 non-null  object
 4   year_posted         49186 non-null  int64 
 5   sentiment_analysis  49186 non-null  int64 
dtypes: bool(1), int64(2), object(3)
memory usage: 1.9+ MB


,user_id,item_id,recommend,review,year_posted,sentiment_analysis
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,1
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011,1
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,1
3,js41637,251610,True,I know what you think when you see this title ...,2014,1
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013,1


In [20]:
df_UsersRecommend = df_reviews_recomend[['item_id','recommend','year_posted']].copy()
df_UsersRecommend = df_UsersRecommend[df_UsersRecommend['recommend']]

df_UsersRecommend['Recomends_times'] = df_UsersRecommend.groupby(['item_id','year_posted'])['item_id'].transform('count')
df_UsersRecommend.drop_duplicates(subset=['item_id','year_posted','Recomends_times'],inplace=True)
df_UsersRecommend.reset_index(drop=True,inplace=True)

df_UsersRecommend.sort_values(by='Recomends_times',ascending=False,inplace=True)
df_UsersRecommend.head()

,item_id,recommend,year_posted,Recomends_times
26,440,True,2014,1564
38,730,True,2015,1551
19,730,True,2014,1079
71,440,True,2013,798
32,4000,True,2014,764


In [21]:
# Funcion de consulta
def UsersRecommend(anio):
    top_recomend = df_UsersRecommend[df_UsersRecommend['year_posted'] == anio]
    return top_recomend['item_id'].head(3)

In [22]:
UsersRecommend(2014)

26     440
19     730
32    4000
Name: item_id, dtype: object

In [23]:
# Funcion de consulta
def UsersNotRecommend(anio):
    top_not_recomend = df_UsersRecommend[df_UsersRecommend['year_posted'] == anio]
    return top_not_recomend['item_id'].tail(3)

In [24]:
UsersNotRecommend(2014)

2762    264930
2758    268830
2757    233350
Name: item_id, dtype: object

<b>sentiment_analysis:</b>       

Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

In [25]:
df_reviews.head()

,user_id,item_id,recommend,review,year_posted,sentiment_analysis
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,1
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011,1
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,1
3,js41637,251610,True,I know what you think when you see this title ...,2014,1
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013,1


In [26]:
df_SentimentRecommend = df_reviews[['item_id','year_posted','sentiment_analysis']].copy()

# Agrupando por 'year_posted','sentiment_analysis' y contando la cantidad de datos
df_SentimentRecommend['Recomends_times'] = df_SentimentRecommend.groupby(['item_id','year_posted','sentiment_analysis'])['year_posted'].transform('count')
df_SentimentRecommend.drop_duplicates(subset=['item_id','year_posted','sentiment_analysis'],inplace=True)
df_SentimentRecommend.sort_values(by='sentiment_analysis',ascending=True,inplace=True)
df_SentimentRecommend.reset_index(drop=True,inplace=True)


In [27]:
# Cambiando contenido de sentiment_analysis
df_SentimentRecommend['sentiment_analysis'].replace(0,'Negative',inplace=True)
df_SentimentRecommend['sentiment_analysis'].replace(1,'Neutral',inplace=True)
df_SentimentRecommend['sentiment_analysis'].replace(2,'Positive',inplace=True)

df_SentimentRecommend

,item_id,year_posted,sentiment_analysis,Recomends_times
0,12210,2014,Negative,3
1,630,2013,Negative,1
2,33950,2014,Negative,1
3,204080,2014,Negative,1
4,322920,2015,Negative,1
...,...,...,...,...
6875,221430,2013,Positive,1
6876,10,2012,Positive,2
6877,49540,2013,Positive,1
6878,42719,2014,Positive,1


In [28]:
df_SentimentRecommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6880 entries, 0 to 6879
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_id             6880 non-null   object
 1   year_posted         6880 non-null   int64 
 2   sentiment_analysis  6880 non-null   object
 3   Recomends_times     6880 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 215.1+ KB


In [29]:
anio = 2014
analisis_sentimiento = df_SentimentRecommend[df_SentimentRecommend['year_posted'] == anio]
analisis_sentimiento.count()

item_id               2374
year_posted           2374
sentiment_analysis    2374
Recomends_times       2374
dtype: int64

In [30]:
def sentiment_analysis(anio):
    analisis_sentimiento = df_SentimentRecommend[df_SentimentRecommend['year_posted'] == anio]

    return print('Negative: ', analisis_sentimiento['sentiment_analysis'][analisis_sentimiento['sentiment_analysis'] == 'Negative'].count(),
                 'Neutral: ', analisis_sentimiento['sentiment_analysis'][analisis_sentimiento['sentiment_analysis'] == 'Neutral'].count(),
                 'Positive: ', analisis_sentimiento['sentiment_analysis'][analisis_sentimiento['sentiment_analysis'] == 'Positive'].count())

In [31]:
sentiment_analysis(2014)

Negative:  223 Neutral:  1512 Positive:  639


<b>Guardar dataframes para Funciones</b>

In [32]:
#df_steam.to_csv('..\\New_datasets\\user_steams.csv',index=False)
df_PlayTimeGenre.to_parquet('..\\Funciones\\datasets\\PlayTimeGenre.parquet',index=False)
#df_UserForGenre_group.to_parquet('..\\Funciones\\datasets\\UserForGenre.parquet',index=False)
df_UserForGenre.to_csv('..\\Funciones\\datasets\\UserForGenre.csv',index=False)
df_UsersRecommend.to_parquet('..\\Funciones\\datasets\\UsersRecommend.parquet',index=False)
df_SentimentRecommend.to_parquet('..\\Funciones\\datasets\\SentimentRecommend.parquet',index=False)